In [1]:
import pandas as pd
import numpy as np
import itertools
import logging

# Visualización de grafos
from pyvis.network import Network

# Interacciones en el notebook
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# Configuro el loggin
logging.basicConfig(level=logging.DEBUG, filename='community-visualization.log', filemode='w')

# Cargo los datos de las interacciones
artworks_data = pd.read_csv('/data/Prado_artworks_wikidata.csv')
interactions_df = pd.read_csv('/data/Prado_emotions.csv')
users_df = pd.read_csv('/data/Prado_users.csv')

# Comunidades de ejemplo (usando emoción Anger)
anger_1 = [262, 306, 132, 247, 33]
anger_2 = [33, 50, 99, 87, 103, 84, 8, 86]
anger_3 = [103, 84, 171, 98, 11, 26, 79]
anger_4 = [26, 11, 59, 117, 175]
anger_5 = [300, 44, 103, 243, 31, 225, 63, 153, 147, 106, 17]
anger_6 = [20, 159, 91, 92]
anger_7 = [16, 149, 140]
communities = [anger_1, anger_2, anger_3, anger_4, anger_5, anger_6, anger_7]

# Creo diccionarios para obtener imagenes y títulos de los cuadros
artwork_ids = np.unique(interactions_df.artworkId.values)
artworks_dict = dict()
images_dict = dict()
for art_id in artwork_ids:
    title = artworks_data[artworks_data['ID'] == art_id]['Title'].values[0]
    image = artworks_data[artworks_data['ID'] == art_id]['Image URL'].values[0]
    artworks_dict[art_id] = title
    images_dict[art_id] = image
    
# Create array of emotions and its colors
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c',
          '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
          '#000075', '#808080']
emotions = np.unique(interactions_df['emotion'].values)
emotions_colors = dict()
for i in range(len(emotions)):
    emotions_colors[emotions[i]] = colors[i]

In [3]:
# Creo una lista de colores para las diferentes comunidades
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0']

# Creo un diccionario de usuarios --> comunidades
users_communities_dict = dict()
communities_colors_dict= {'None':'#EAEAEA'}
communities_names = []
for i in range(len(communities)):
    community_name = 'anger_' + str(i)
    communities_names.append(community_name)
    communities_colors_dict[community_name] = colors[i]
    for user in communities[i]:
        if user in users_communities_dict:
            users_communities_dict[user].append(community_name)
        else:
            users_communities_dict[user] = [community_name]

In [4]:
def get_users_relations(df, emotion=None):
    """
    Función para obtener las relaciones que tienen los usuarios entre sí. Es una lista que, en cada elemento,
    se muestra una lista de los usuarios que han interactuado con la misma emoción (o emoción seleccionada)
    con un mismo cuadro.
    """
    if emotion == None:
        return df.groupby(by=['artworkId', 'emotion'])['userId'].apply(list).values
    else:
        aux_df = df[df['emotion'] == emotion]
        return aux_df.groupby(by=['artworkId', 'emotion'])['userId'].apply(list).values

In [5]:
def get_users_edges(df, emotion=None):
    users_relations = get_users_relations(df, emotion)
    
    # Obtengo todos los pares de relaciones
    relations_pairs = [[pair for pair in itertools.combinations(relations, 2)] for relations in users_relations]
    edges = [edge for relations in relations_pairs for edge in relations]
    
    # Obtengo el peso de cada eje
    edges_dict = dict()
    for edge in edges:
        if edge in edges_dict:
            edges_dict[edge] += 1
        elif (edge[1], edge[0]) in edges_dict:
            edges_dict[(edge[1], edge[0])] += 1
        else:
            edges_dict[edge] = 1
    
    # Creo la lista final de ejes (from, to, weigth)
    user_edges = []
    for i in edges_dict:
        user_edges.append((i[0], i[1], edges_dict[i]))
        
    return user_edges

In [6]:
def get_community_user(userId):
    if userId in users_communities_dict:
        return users_communities_dict[userId]
    else:
        return None

In [7]:
def same_community(user1, user2):
    community_u1 = get_community_user(user1)
    community_u2 = get_community_user(user2)
    
    if community_u1 != None and community_u2 != None:
        return list(set(community_u1).intersection(community_u2))
    else:
        return None

In [8]:
def get_nodes(vis, elements_to_show):
    logging.info('get_nodes')
    all_nodes = np.unique(interactions_df['userId'].values)
    
    for n in all_nodes:
        logging.debug('Node-{}'.format(n))
        
        # Comprobamos su comunidad
        communities = None
        color_community = communities_colors_dict['None']
        
        if n in users_communities_dict:
            communities = users_communities_dict[n]
            color_community = communities_colors_dict[communities[0]]
            logging.debug('Communities: {}'.format(communities))
        else:
            logging.debug('No Community')
            
        
        if elements_to_show == 'All':
            # Lo incluimos sin más
            vis.add_node(str(n), color=color_community, size=3)
            logging.debug('Added in All')
        else:
            if communities != None:
                if elements_to_show == 'Only Communities':
                    # Solo incluimos el nodo si pertenece a una comunidad
                    if communities != None:
                        vis.add_node(str(n), color=color_community, size=3)
                        logging.debug('Added in Only Communities')

                else:

                    if elements_to_show in communities:
                        color_community = communities_colors_dict[elements_to_show]
                        vis.add_node(str(n), color=color_community, size=3)
                        logging.debug('Added in {}'.format(elements_to_show))
                
    return vis   

In [9]:
def get_edges(vis, emotion, elements_to_show):
    all_edges = get_users_edges(interactions_df, emotion)
    
    for e in all_edges:
        
        common_communities = same_community(e[0], e[1])
        edge_color = communities_colors_dict['None']
        
        if common_communities != None and len(common_communities) > 0:
            edge_color = communities_colors_dict[common_communities[0]]
           
        if elements_to_show == 'Only Communities' and common_communities != None and len(common_communities) > 0:
            vis.add_edge(str(e[0]), str(e[1]), color=edge_color, value=e[2])
        elif elements_to_show == 'All':
            vis.add_edge(str(e[0]), str(e[1]), color=edge_color, value=e[2])
        else:
            if common_communities != None and elements_to_show in common_communities:
                edge_color = communities_colors_dict[elements_to_show]
                vis.add_edge(str(e[0]), str(e[1]), color=edge_color, value=e[2])
                
    return vis 

In [10]:
def show_graph_users(emotion=None, show_physics=False, elements_to_show='All'):
    nodes = np.unique(interactions_df['userId'].values)
    edges = get_users_edges(interactions_df, emotion)

    vis = Network(notebook=True, width="1000px", height="800px")
    '''
    for n in nodes:
        color_community = communities_colors_dict['None']
        if n in users_communities_dict:
            color_community = communities_colors_dict[users_communities_dict[n][0]]
        vis.add_node(str(n), color=color_community, size=3)
    '''
    vis = get_nodes(vis, elements_to_show)
    '''
    for e in edges:
        common_communities = same_community(e[0], e[1])
        edge_color = communities_colors_dict['None']
        if common_communities != None and len(common_communities) > 0:
            edge_color = communities_colors_dict[common_communities[0]]
        vis.add_edge(str(e[0]), str(e[1]), color=edge_color, value=e[2])
    '''
    vis = get_edges(vis, emotion, elements_to_show)
    if show_physics:
        vis.show_buttons(filter_=['physics'])
        
    return vis.show('users_graph.html')

In [11]:
elements_to_show = ['All', 'Only Communities']
elements_to_show += communities_names

In [12]:
emotions_options = list(emotions)
emotions_options.append(None)

In [13]:
@interact_manual
def show_graphs(nodes=['artworks', 'users'], emotion=emotions_options, elements=elements_to_show, show_physics=False):
    return show_graph_users(emotion, show_physics, elements)
    '''
    if nodes == 'artworks':
        return show_graph_artworks(emotion, show_physics)
    else:
        return show_graph_users(emotion, show_physics)
    '''

interactive(children=(Dropdown(description='nodes', options=('artworks', 'users'), value='artworks'), Dropdown…